In [9]:
import transformers
import torch
import threading

# Example model IDs for illustration:
#   - 1) A "language" or "chat" transformer
#   - 2) An "action" or "instruction" transformer
model_id_language = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id_action   = "meta-llama/Meta-Llama-3-8B-Instruct"  # Placeholder: use another model if desired

# Create two separate pipelines
pipeline_lang = transformers.pipeline(
    "text-generation",
    model=model_id_language,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

pipeline_action = transformers.pipeline(
    "text-generation",
    model=model_id_action,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

# Example messages (the same or different, depending on your use case)
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user",   "content": "Who are you?"},
]

# Terminators can be pipeline-specific but we will just reuse for simplicity
terminators = [
    pipeline_lang.tokenizer.eos_token_id,
    pipeline_lang.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

def generate_response(pipeline, prompt, terminators, result_container, index):
    """
    Thread worker function that runs the pipeline and
    stores the generated text in `result_container[index]`.
    """
    outputs = pipeline(
        prompt,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    # Store the entire text or just the appended portion; your choice:
    result_container[index] = outputs[0]["generated_text"]


def main():
    # Container for both results
    results = [None, None]

    # Create threads for parallel calls
    t_lang = threading.Thread(
        target=generate_response,
        args=(pipeline_lang, messages, terminators, results, 0)
    )
    t_action = threading.Thread(
        target=generate_response,
        args=(pipeline_action, messages, terminators, results, 1)
    )

    # Start threads
    t_lang.start()
    t_action.start()

    # Wait for both to complete
    t_lang.join()
    t_action.join()

    # Now we have two outputs from two different pipelines
    print("=== Language Transformer Output ===")
    print(results[0])
    print("\n=== Action Transformer Output ===")
    print(results[1])

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'torch'

In [10]:
import threading
import queue
import time

# 1) transformers 가 설치 안 되어 있으면 아래 주석 해제 후 실행
# !pip install transformers

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TextIteratorStreamer
)
import torch

def stream_generation(model, tokenizer, prompt, token_queue, done_event):
    """
    모델에 prompt를 넣어 실시간(스트리밍)으로 토큰을 생성한다.
    새로 생성된 토큰은 token_queue 에 넣는다.
    생성이 끝나면 done_event.set()으로 알린다.
    """
    # TextIteratorStreamer: 토큰 생성 시점마다 yield 해줌
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True)
    
    # 텍스트를 텐서로 변환
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # generate()에 필요한 파라미터
    generation_kwargs = {
        "max_new_tokens": 50,  # 필요에 맞춰 조절
        "do_sample": True,
        "top_p": 0.9,
        "temperature": 0.6,
        "streamer": streamer
    }
    generation_kwargs.update(inputs)

    # 모델이 토큰을 계속 생성하는 부분을 별도 스레드에서 돌린다.
    def _generate():
        model.generate(**generation_kwargs)
        done_event.set()  # 생성 종료를 알림

    t = threading.Thread(target=_generate)
    t.start()

    # streamer 에서 새 토큰이 yield 될 때마다 queue에 넣는다.
    for new_text in streamer:
        token_queue.put(new_text)

    t.join()  # 생성이 완전히 끝날 때까지 대기
    done_event.set()

def display_generation(role_name, token_queue, done_event):
    """
    token_queue로 들어오는 토큰을 실시간으로 꺼내 출력한다.
    done_event가 set 되면 종료한다.
    """
    print(f"=== {role_name} ===", flush=True)
    while not done_event.is_set() or not token_queue.empty():
        try:
            # 토큰이 들어올 때까지 0.1초 대기
            new_text = token_queue.get(timeout=0.1)
            print(new_text, end="", flush=True)
        except:
            pass
    print("\n")  # 한 줄 띄우고 종료

def main():
    # ModuleNotFoundError 에 대한 안내
    print("만약 `ModuleNotFoundError: No module named 'transformers'` 오류가 나면:")
    print("!pip install transformers\n")

    # -----------------------------
    # (1) 모델/토크나이저 로드
    # -----------------------------
    # 인증이 필요 없는 공개 모델 2종을 사용합니다.
    #  - model_lang_id: "gpt2"
    #  - model_act_id : "microsoft/DialoGPT-medium"
    model_lang_id = "gpt2"
    model_act_id  = "microsoft/DialoGPT-medium"

    print("모델을 불러오는 중입니다... (CPU에서는 다소 시간이 걸릴 수 있음)\n")
    model_lang = AutoModelForCausalLM.from_pretrained(model_lang_id)
    tokenizer_lang = AutoTokenizer.from_pretrained(model_lang_id)

    model_act = AutoModelForCausalLM.from_pretrained(model_act_id)
    tokenizer_act = AutoTokenizer.from_pretrained(model_act_id)

    # GPU가 있으면 GPU 사용 (없으면 CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_lang.to(device)
    model_act.to(device)

    # -----------------------------
    # (2) 사용자 프롬프트 설정
    # -----------------------------
    # 간단한 예시로 "Who are you?" 를 사용.
    prompt = "Who are you?\n"

    # -----------------------------
    # (3) 병렬 스레드로 스트리밍 출력
    # -----------------------------
    # 각 모델별로 queue / event를 만들어준다
    queue_lang = queue.Queue()
    queue_act  = queue.Queue()

    done_event_lang = threading.Event()
    done_event_act  = threading.Event()

    # generate + streamer 동작 스레드
    t_gen_lang = threading.Thread(target=stream_generation,
                                  args=(model_lang, tokenizer_lang, prompt, queue_lang, done_event_lang))
    t_gen_act  = threading.Thread(target=stream_generation,
                                  args=(model_act, tokenizer_act, prompt, queue_act, done_event_act))

    # 실시간 출력 스레드
    t_disp_lang = threading.Thread(target=display_generation, 
                                   args=("Language Transformer", queue_lang, done_event_lang))
    t_disp_act  = threading.Thread(target=display_generation, 
                                   args=("Action Transformer", queue_act, done_event_act))

    print("생성을 시작합니다...\n")

    # 스레드 시작
    t_gen_lang.start()
    t_gen_act.start()
    t_disp_lang.start()
    t_disp_act.start()

    # 모든 스레드가 끝날 때까지 대기
    t_gen_lang.join()
    t_gen_act.join()
    t_disp_lang.join()
    t_disp_act.join()

    print("=== 모든 생성이 완료되었습니다. ===")

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'torch'

In [12]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install torch

  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 9.6 MB/s eta 0:00:0000:0100:01
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install sentencepiece


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install protobuf

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import threading
import queue
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    TextIteratorStreamer,
)

#############################################
# 1) 두 개의 다른 모델(혹은 동일 모델)을 준비
#############################################

# 여기서는 예시로 동일한 skt/kogpt2-base-v2 모델을
# "언어적 출력용" / "행동적 출력용" 두 번 로드한다고 가정합니다.

MODEL_NAME_1 = "skt/kogpt2-base-v2"  # 언어적 출력용
MODEL_NAME_2 = "skt/kogpt2-base-v2"  # 행동적 출력용 (예시상 동일 모델)

# (필요시) 캐시 경로 설정
os.environ["TRANSFORMERS_CACHE"] = "./cache/"
os.environ["HF_HOME"] = "./cache/"

print("=== 모델/토크나이저 로딩 중... 시간이 다소 걸릴 수 있습니다. ===")

# 토크나이저 / 모델 로드 (모델1)
tokenizer_1 = AutoTokenizer.from_pretrained(MODEL_NAME_1)
model_1 = AutoModelForCausalLM.from_pretrained(MODEL_NAME_1)
streamer_1 = TextIteratorStreamer(tokenizer_1, skip_prompt=True, skip_special_tokens=True)

pipeline_1 = pipeline(
    task="text-generation",
    model=model_1,
    tokenizer=tokenizer_1,
    device=-1,            # CPU 사용: -1 / GPU 사용: 0
    max_new_tokens=128,   # 생성할 토큰 최대 개수
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    streamer=streamer_1,  # <-- 스트리머 장착 (실시간 토큰 생성)
)

# 토크나이저 / 모델 로드 (모델2)
tokenizer_2 = AutoTokenizer.from_pretrained(MODEL_NAME_2)
model_2 = AutoModelForCausalLM.from_pretrained(MODEL_NAME_2)
streamer_2 = TextIteratorStreamer(tokenizer_2, skip_prompt=True, skip_special_tokens=True)

pipeline_2 = pipeline(
    task="text-generation",
    model=model_2,
    tokenizer=tokenizer_2,
    device=-1,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    streamer=streamer_2,
)

print("=== 모델/토크나이저 로딩 완료! ===\n")


#############################################
# 2) 스레드에서 모델 추론 + 스트리밍 출력
#############################################

def generate_stream(
    pipe, 
    prompt: str, 
    streamer: TextIteratorStreamer, 
    output_queue: queue.Queue,
    prefix_name: str = "",
):
    """
    특정 pipeline(모델)에 prompt를 넣고,
    스트리밍으로 토큰이 생성될 때마다 print로 찍어주는 함수.
    최종 출력은 output_queue에 put()하여 반환.
    """
    # 1) prompt를 주어 추론 시작 (비동기적으로 토큰 생성)
    _ = pipe(prompt)

    # 2) 스트리머에서 토큰을 하나씩 뽑아가며 바로 print
    text_buffer = ""
    for new_token in streamer:
        text_buffer += new_token
        print(f"{prefix_name}{new_token}", end="", flush=True)

    # 3) 최종 결과 문자열을 Queue에 넣음
    output_queue.put(text_buffer)


def main():
    # 예시 입력(하나)
    user_input = "대한민국의 수도와 간단한 역사적 배경을 알려주고,\n" \
                 "이 내용에 기반해 오늘 할 수 있는 액티비티를 추천해줘."

    # 1) 언어적 출력용 프롬프트
    prompt_for_language = (
        "다음 질문에 대해 친절하고 자세한 한국어 답변을 작성해줘:\n\n"
        f"질문: {user_input}\n\n답변:"
    )
    # 2) 행동(액션) 출력용 프롬프트
    prompt_for_action = (
        "당신은 여행 가이드처럼 행동해야 합니다.\n"
        "질문을 보고, 구체적인 액션 플랜을 단계별로 제시하세요.\n\n"
        f"질문: {user_input}\n\n행동 계획:"
    )

    # 두 개의 Queue: 각 스레드(모델)에서 최종 text를 담아놓을 곳
    output_queue_1 = queue.Queue()
    output_queue_2 = queue.Queue()

    print("=== 두 모델에서 동시에 스트리밍 답변을 시작합니다. ===\n")

    # 스레드 생성
    t1 = threading.Thread(
        target=generate_stream,
        args=(pipeline_1, prompt_for_language, streamer_1, output_queue_1, "[언어출력] ")
    )
    t2 = threading.Thread(
        target=generate_stream,
        args=(pipeline_2, prompt_for_action, streamer_2, output_queue_2, "[행동출력] ")
    )

    # 스레드 시작
    t1.start()
    t2.start()

    # 두 스레드 종료 대기
    t1.join()
    t2.join()

    # 최종 결과 받기
    lang_result = output_queue_1.get()
    action_result = output_queue_2.get()

    print("\n\n=== 최종 결과 정리 ===\n")
    print("[언어적 출력]\n", lang_result, "\n")
    print("[행동적 출력]\n", action_result, "\n")


if __name__ == "__main__":
    main()


=== 모델/토크나이저 로딩 중... 시간이 다소 걸릴 수 있습니다. ===


Device set to use cpu
/Users/jeongsugwang/.pyenv/versions/3.9.13/lib/python3.9/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Device set to use cpu


=== 모델/토크나이저 로딩 완료! ===

=== 두 모델에서 동시에 스트리밍 답변을 시작합니다. ===



/Users/jeongsugwang/.pyenv/versions/3.9.13/lib/python3.9/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


[언어출력] [언어출력] 한국 [언어출력] [언어출력] 내에서도 [언어출력] 널리 [언어출력] 알려진 [언어출력] 한국의 [언어출력] [언어출력] 수도와 [언어출력] 간단한 [언어출력] 역사적 [언어출력] 배경을 [언어출력] [언어출력] 알려주고,
[언어출력] [언어출력] [언어출력] [언어출력] 답변: [언어출력] 한국의 [언어출력] [언어출력] 수도와 [언어출력] 주요 [언어출력] 역사적 [언어출력] 배경을 [언어출력] [언어출력] 알려주고,
[언어출력] [언어출력] [언어출력] [언어출력] 답변: [언어출력] 한국 [언어출력] [언어출력] 내에서도 [언어출력] 널리 [언어출력] 알려진 [언어출력] 한국 [언어출력] [언어출력] 수도와 [언어출력] 주요 [언어출력] 역사적 [언어출력] 배경을 [언어출력] [언어출력] 알려주고,
[언어출력] [언어출력] [언어출력] [언어출력] 답변: [언어출력] 한국의 [언어출력] [언어출력] 수도와 [언어출력] 주요 [언어출력] 역사적 [언어출력] 배경을 [언어출력] [언어출력] 알려주고,
[언어출력] [언어출력] [언어출력] [언어출력] 답변: [언어출력] 한국 [언어출력] [언어출력] 내에서도 [언어출력] 널리 [언어출력] 알려진 [언어출력] 한국 [언어출력] [언어출력] 수도와 [언어출력] 주요 [언어출력] 역사적 [언어출력] 배경을 [언어출력] [언어출력] 알려주고,
[언어출력] [언어출력] [언어출력] [언어출력] 답변: [언어출력] 한국 [언어출력] [언어출력] 내에서도 [언어출력] 널리 [언어출력] 알려진 [언어출력] 한국 [언어출력] [언어출력] 수도와 [언어출력] 주요 [언어출력] 역사적 [언어출력] 배경을 [언어출력] [언어출력] 알려주고,
[언어출력] [언어출력] [언어출력] [언어출력] 답변: [언어출력] 한국의 [언어출력] [언어출력] 수도와 [언어출력] 주요 [언어출력] 역사적 [언어출력] 배경을 [언어출력] [언어출력] 알려주고,
[언어출력] [언어출력] [언어출력] [언어출력] 답변: [언어출

In [3]:
import threading
import ollama

def pull_models():
    """
    모델을 Pull하여 설치하는 함수
    """
    models_to_pull = ["gemma", "mistral"]
    try:
        models_info = ollama.list()
        print(f"ollama.list() 반환 값: {models_info}")  # 디버깅용 출력

        # 'models_info'가 'models'라는 속성을 가지고 있다고 가정
        models_list = models_info.models

        # 모델 리스트가 Model 객체의 리스트라고 가정
        installed_models = [model.model for model in models_list]

        for model in models_to_pull:
            if model not in installed_models:
                print(f"모델 '{model}'을(를) Pull 중입니다...")
                try:
                    ollama.pull(model)
                    print(f"모델 '{model}'이(가) 성공적으로 설치되었습니다.")
                except Exception as e:
                    print(f"모델 '{model}'을(를) Pull하는 중 오류가 발생했습니다: {e}")
            else:
                print(f"모델 '{model}'은(는) 이미 설치되어 있습니다.")
    except Exception as e:
        print(f"모델 목록을 가져오는 중 오류가 발생했습니다: {e}")

def stream_language_output(user_prompt):
    """
    첫 번째 출력(언어적 표현)에 해당하는 모델 스트리밍 함수
    """
    print("\n[언어적 출력 시작]\n")
    try:
        for chunk in ollama.generate(
            model="gemma",  # 실제 사용하실 모델명
            prompt=user_prompt,
            stream=True,
            system="""
            너는 한국어로 답변을 작성하는 '언어적' 모델이야.
            """,
        ):
            # chunk는 {"completion": "..."} 같은 형태로 토큰이 생성
            print(chunk["completion"], end="", flush=True)
        print("\n[언어적 출력 종료]\n")
    except Exception as e:
        print(f"[언어적 출력 오류]: {e}\n")

def stream_action_output(user_prompt):
    """
    두 번째 출력(행동적 표현)에 해당하는 모델 스트리밍 함수
    """
    print("\n[행동적 출력 시작]\n")
    try:
        for chunk in ollama.generate(
            model="mistral",  # 실제 사용하실 모델명
            prompt=user_prompt,
            stream=True,
            system="""
            너는 특정 행동이나 단계(step-by-step action)를 설명하는 '행동적' 모델이야.
            """,
        ):
            print(chunk["completion"], end="", flush=True)
        print("\n[행동적 출력 종료]\n")
    except Exception as e:
        print(f"[행동적 출력 오류]: {e}\n")

def main():
    # 모델 Pull 및 설치
    pull_models()

    user_input = input("사용자 입력을 작성해주세요: ")

    # 두 개의 스레드를 각각 실행
    t1 = threading.Thread(target=stream_language_output, args=(user_input,))
    t2 = threading.Thread(target=stream_action_output, args=(user_input,))

    # 스레드 시작
    t1.start()
    t2.start()

    # 두 스레드가 모두 끝날 때까지 대기
    t1.join()
    t2.join()

    print("\n모든 스트리밍이 완료되었습니다.")

if __name__ == "__main__":
    main()


ollama.list() 반환 값: models=[Model(model='llama3.2:latest', modified_at=datetime.datetime(2024, 10, 9, 2, 3, 50, 841424, tzinfo=TzInfo(+09:00)), digest='a80c4f17acd55265feec403c7aef86be0c25983ab279d83f3bcd3abbcb5b8b72', size=2019393189, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='3.2B', quantization_level='Q4_K_M'))]
모델 목록을 가져오는 중 오류가 발생했습니다: 0

[언어적 출력 시작]

[행동적 출력 시작]


[언어적 출력 오류]: model "gemma" not found, try pulling it first

[행동적 출력 오류]: model "mistral" not found, try pulling it first


모든 스트리밍이 완료되었습니다.


In [6]:
import ollama
ollama.pull('llama2')

KeyboardInterrupt: 

In [8]:
ollama.list()

ListResponse(models=[Model(model='llama3.2:latest', modified_at=datetime.datetime(2024, 10, 9, 2, 3, 50, 841424, tzinfo=TzInfo(+09:00)), digest='a80c4f17acd55265feec403c7aef86be0c25983ab279d83f3bcd3abbcb5b8b72', size=2019393189, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='3.2B', quantization_level='Q4_K_M'))])

In [ ]:
response = ollama.chat(model='llama3.2:latest', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh. He discovered that shorter wavelengths of light are scattered more by tiny molecules in the atmosphere than longer wavelengths.

Here's what happens:

1. When sunlight enters Earth's atmosphere, it encounters tiny molecules like nitrogen (N2) and oxygen (O2).
2. These molecules scatter the light in all directions, but they scatter shorter wavelengths (like blue and violet) more than longer wavelengths (like red and orange).
3. The scattered blue light is then dispersed throughout the sky, giving it a blue appearance.
4. Our eyes perceive this scattering of blue light as the color of the sky.

Now, you might be wondering why the sky isn't just a deep shade of purple, since violet light is scattered even more than blue light. That's because our eyes are more sensitive to blue light, and the human eye has a built-in bias towards perceiving blues and greens rather than 

In [10]:
import ollama

def handle_streaming_response(response_stream):
    for chunk in response_stream:
        # 각 청크를 디코딩하여 출력
        if 'content' in chunk['message']:
            print(chunk['message']['content'], end='', flush=True)

# 채팅 요청을 스트리밍 모드로 설정
response = ollama.chat(
    model='llama3.2:latest',
    messages=[
        {
            'role': 'user',
            'content': 'Why is the sky blue?',
        },
    ],
    stream=True  # 스트리밍 활성화
)

# 스트리밍 응답 처리
handle_streaming_response(response)


The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh. He discovered that when sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen and oxygen.

Here's what happens:

1. Sunlight is made up of different colors, like a rainbow.
2. When sunlight enters the atmosphere, these colors are scattered in all directions by the tiny molecules.
3. The shorter (blue) wavelengths are scattered more than the longer (red) wavelengths because they are more easily deflected by the smaller molecules.

Think of it like this: imagine you're at the beach and throw a blue ball into the air. What happens? It scatters in all directions, right? That's similar to what's happening with sunlight and its colors when it enters our atmosphere!

As a result of this scattering, the blue light is dispersed throughout the sky, making it appear blue to our eyes. The color we see can also change depending on the time of

In [2]:
import threading
import ollama

def handle_streaming_response(prefix, response_stream):
    """
    스트리밍 응답을 실시간으로 출력하는 함수.
    """
    try:
        for chunk in response_stream:
            if 'content' in chunk['message']:
                print(f"{prefix}: {chunk['message']['content']}", end='', flush=True)
    except Exception as e:
        print(f"{prefix}: 에러 발생 - {e}")

def get_linguistic_response(model, user_input):
    """
    언어적 답변을 얻기 위한 함수.
    """
    prompt = f"Provide a detailed linguistic explanation for the following question:\n{user_input}"
    response = ollama.chat(
        model=model,
        messages=[
            {
                'role': 'user',
                'content': prompt,
            },
        ],
        stream=True  # 스트리밍 활성화
    )
    handle_streaming_response("언어적 답변", response)

def get_action_response(model, user_input):
    """
    행동적 답변을 얻기 위한 함수.
    """
    prompt = f"Provide actionable steps based on the following question:\n{user_input}"
    response = ollama.chat(
        model=model,
        messages=[
            {
                'role': 'user',
                'content': prompt,
            },
        ],
        stream=True  # 스트리밍 활성화
    )
    handle_streaming_response("행동적 답변", response)

def main():
    user_input = "Why is the sky blue?"
    model = 'llama3.2:latest'

    # 두 개의 스레드를 생성하여 병렬로 실행
    thread1 = threading.Thread(target=get_linguistic_response, args=(model, user_input))
    thread2 = threading.Thread(target=get_action_response, args=(model, user_input))

    thread1.start()
    thread2.start()

    thread1.join()
    thread2.join()

if __name__ == "__main__":
    main()


행동적 답변: Here언어적 답변: What행동적 답변:  are언어적 답변:  a행동적 답변:  some언어적 답변:  de행동적 답변:  actionable언어적 답변: cept행동적 답변:  steps언어적 답변: ively행동적 답변:  to언어적 답변:  simple행동적 답변:  explore언어적 답변:  question행동적 답변:  why언어적 답변: !행동적 답변:  the언어적 답변:  The행동적 답변:  sky언어적 답변:  inquiry행동적 답변:  appears언어적 답변:  "행동적 답변:  blue언어적 답변: Why행동적 답변: :

언어적 답변:  is행동적 답변: **언어적 답변:  the행동적 답변: Step언어적 답변:  sky행동적 답변:  언어적 답변:  blue행동적 답변: 1언어적 답변: ?"행동적 답변: :언어적 답변:  del행동적 답변:  Ob언어적 답변: ves행동적 답변: serve언어적 답변:  into행동적 답변:  and언어적 답변:  the행동적 답변:  Note언어적 답변:  realm행동적 답변:  Your언어적 답변:  of행동적 답변:  Surround언어적 답변:  physics행동적 답변: ings언어적 답변: ,행동적 답변: **

언어적 답변:  atmospheric행동적 답변: *언어적 답변:  science행동적 답변:  Find언어적 답변: ,행동적 답변:  a언어적 답변:  and행동적 답변:  comfortable언어적 답변:  lingu행동적 답변:  spot언어적 답변: istics행동적 답변:  outside언어적 답변: .행동적 답변:  with언어적 답변:  Let행동적 답변:  minimal언어적 답변: 's행동적 답변:  ob언어적 답변:  dissect행동적 답변: structions언어적 답변:  this행동적 답변:  (언어적 답변:  query행동적 답변: e언어적 답변:  through행동적 답변: .g언어적 답변:  a행동적 답변: .,언어적 답변: 

In [7]:
pip install accelerate>=0.26.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install nest_asyncio


Note: you may need to restart the kernel to use updated packages.
